In [1]:
import os

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1-20240815_150139.xlsx',
 'crawler_10-20240815_150146.xlsx',
 'crawler_11-20240815_150201.xlsx',
 'crawler_12-20240815_150212.xlsx',
 'crawler_13-20240815_150247.xlsx',
 'crawler_14-20240815_150249.xlsx',
 'crawler_15-20240815_150302.xlsx',
 'crawler_16-20240815_150316.xlsx',
 'crawler_17-20240815_150318.xlsx']

In [2]:
file = list_file[1]

file

'crawler_10-20240815_150146.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/3.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)
input_['rowId'] = input_['rowId'].astype(int)
input_['qtyEach'] = input_['qtyEach'].astype(int)
input_['maxRows'] = input_['maxRows'].astype(int)

print(f'总数量：{len(input_)}')
input_['Part Number'].tolist()

总数量：12


['53021',
 '55030',
 '55032',
 '55046',
 '55052',
 '56266',
 '56268',
 '56270',
 '56980',
 '56986',
 '56988',
 '56992']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(f'''https://ecatalog.smpcorp.com/v2/fs/api/image/getallimages?partNum={input_.loc[a, 'Part Number']}&brand={input_.loc[a, 'brand']}&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960''',
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


<Response [200]>

In [8]:
list_src = resp.json()
list_src = [src['image_BG_Url'].strip() for src in list_src]

list_src

['https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=/GiWvhhYsuWkKHfMNsqxKSYF4nkLByFFki+PA/U85oaueUPW6nd8Su72HZkm08flMYvVb8ruU/a2jlDyZ/9SFhHKSDnV/vfUm++2cf2MyJSTHiO+uZMQDMRJmK/E5kGqgMS+q/NNn547DcAIwETg0+MA4lw7L3WsmesJ/OQLlNQ42zcy8WmfSA==']

In [9]:
dict_src = {}
for i, src in zip(range(len(list_src)), list_src):
    dict_src[str(i)] = src

dict_src

{'0': 'https://ecatalog.smpcorp.com/ihgs/imagehandlergs.ashx?p1=/GiWvhhYsuWkKHfMNsqxKSYF4nkLByFFki+PA/U85oaueUPW6nd8Su72HZkm08flMYvVb8ruU/a2jlDyZ/9SFhHKSDnV/vfUm++2cf2MyJSTHiO+uZMQDMRJmK/E5kGqgMS+q/NNn547DcAIwETg0+MA4lw7L3WsmesJ/OQLlNQ42zcy8WmfSA=='}

In [10]:
import json

df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)

df_temp['Json_Src'] = json.dumps(dict_src)

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Part Number,rowId,site,webBase,brand,basePart,partDesc_en,partLongDesc_en,pop,...,mfG_Part,maxRows,brandLink,imageUrl,Picture,Url,Json_Vehicle,Json_Service,Json_Spec,Json_Src
0,1,53021,1,FS,53021,FS,53021,Cooler Replacement Hose,Cooler Replacement Hose,F,...,,1,,,,https://www.4s.com/en/ecatalog?part=53021&type...,{},"{""0"": ""Fuel Filter Flush (Required)"", ""1"": ""Oi...","{""0"": {""Clamps Required?"": ""No""}, ""1"": {""Descr...","{""0"": ""https://ecatalog.smpcorp.com/ihgs/image..."


In [11]:
crawler_status = 'ok'

crawler_status

'ok'

In [12]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [13]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_src.xlsx', index=False)

output_correct

,No,Part Number,rowId,site,webBase,brand,basePart,partDesc_en,partLongDesc_en,pop,...,mfG_Part,maxRows,brandLink,imageUrl,Picture,Url,Json_Vehicle,Json_Service,Json_Spec,Json_Src
0,1,53021,1,FS,53021,FS,53021,Cooler Replacement Hose,Cooler Replacement Hose,F,...,,1,,,,https://www.4s.com/en/ecatalog?part=53021&type...,{},"{""0"": ""Fuel Filter Flush (Required)"", ""1"": ""Oi...","{""0"": {""Clamps Required?"": ""No""}, ""1"": {""Descr...","{""0"": ""https://ecatalog.smpcorp.com/ihgs/image..."


In [14]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_src_error.xlsx', index=False)

output_error

""
